# Data Cleaning Project: Audible Dataset
## Introduction
In this project I aim to go through the process of data cleaning of the [Audible dataset](https://www.kaggle.com/datasets/snehangsude/audible-dataset) where I address common data issues like duplicates, inconsistent formatting, missing values, and more. The data represents the important details of audiobooks from 1998 till 2025 (pre-planned releases).

### Column Breakdown

- `name`: Name of the audiobook
- `author`: Author of the audiobook
- `narrator`: Narrator of the audiobook
- `time`: Length of the audiobook
- `releasedate`: Release date of the audiobook
- `language`: Language of the audiobook
- `stars`: No. of stars the audiobook received
- `price`: Price of the audiobook in INR
- `ratings`: No. of reviews received by the audiobook


### Step 1. Loading the Dataset and Initial Data Inspectation

In [379]:
import pandas as pd
audible = pd.read_csv('audible_uncleaned.csv')
audible.head()

,name,author,narrator,time,releasedate,language,stars,price
0,Geronimo Stilton #11 & #12,Writtenby:GeronimoStilton,Narratedby:BillLobely,2 hrs and 20 mins,04-08-08,English,5 out of 5 stars34 ratings,468.00
1,The Burning Maze,Writtenby:RickRiordan,Narratedby:RobbieDaymond,13 hrs and 8 mins,01-05-18,English,4.5 out of 5 stars41 ratings,820.00
2,The Deep End,Writtenby:JeffKinney,Narratedby:DanRussell,2 hrs and 3 mins,06-11-20,English,4.5 out of 5 stars38 ratings,410.00
3,Daughter of the Deep,Writtenby:RickRiordan,Narratedby:SoneelaNankani,11 hrs and 16 mins,05-10-21,English,4.5 out of 5 stars12 ratings,615.00
4,"The Lightning Thief: Percy Jackson, Book 1",Writtenby:RickRiordan,Narratedby:JesseBernstein,10 hrs,13-01-10,English,4.5 out of 5 stars181 ratings,820.00


In [380]:
audible.describe(include='all')

,name,author,narrator,time,releasedate,language,stars,price
count,87489,87489,87489,87489,87489,87489,87489,87489
unique,82767,48374,29717,2284,5058,36,665,1011
top,The Art of War,"Writtenby:矢島雅弘,石橋遊",Narratedby:anonymous,2 mins,16-05-18,English,Not rated yet,586.00
freq,20,874,1034,372,773,61884,72417,5533


In [381]:
audible.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 87489 entries, 0 to 87488
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   name         87489 non-null  object
 1   author       87489 non-null  object
 2   narrator     87489 non-null  object
 3   time         87489 non-null  object
 4   releasedate  87489 non-null  object
 5   language     87489 non-null  object
 6   stars        87489 non-null  object
 7   price        87489 non-null  object
dtypes: object(8)
memory usage: 5.3+ MB


In [410]:
audible.isnull().sum()

name                   0
author                 0
narrator               0
language               0
stars                  0
duration_in_minutes    0
release_date           0
ratings                0
price                  0
dtype: int64

**Plan for Data Cleaning:**
1. Check for **duplicates** in the data.
2. Clean the 'author' column: Remove 'Writtenby:' and 'Narratedby:' prefixes, add spaces between name, middle, and last name, and handle multiple persons mentioned.
3. Improve 'time' column: Convert to numeric or datetime format for better analysis.
4. Refactor 'releasedate' column: Check for inconsistency and convert to the `date` data type.
5. Ensure consistency in 'language' column: Capitalize uniformly, check for misspellings, and remove additional characters.
6. Split 'stars' column into 'stars' and 'rating': Store in numeric format for analysis.
7. Investigate 'price' column format: Change to `integer` or `float` as necessary; understand why it's stored as an `object`.


### Step 2. Check Duplicates

In [382]:
audible.duplicated().sum()

0

### Step 3. Clean the 'author' column

In [383]:
# remove the prefix "Writtenby:" and add space between name, middle, and last name of Authors
audible['author'] = audible['author'].str.replace('Writtenby:', '', case=False).str.replace(r"(\w)([A-Z])", r"\1 \2", regex = True)

# remove the prefix "Narratedby:" and add space between name, middle, and last name of Narrators
audible['narrator'] = audible['narrator'].str.replace('Narratedby:', '', case=False).str.replace(r"(\w)([A-Z])", r"\1 \2", regex = True)


In [384]:
# preview the data
audible.loc[:10,['author', 'narrator']]

,author,narrator
0,Geronimo Stilton,Bill Lobely
1,Rick Riordan,Robbie Daymond
2,Jeff Kinney,Dan Russell
3,Rick Riordan,Soneela Nankani
4,Rick Riordan,Jesse Bernstein
5,Suzanne Collins,Tatiana Maslany
6,Winter Morgan,Luke Daniels
7,Rick Riordan,Robbie Daymond
8,Mary Pope Osborne,Mary Pope Osborne
9,Rick Riordan,Robbie Daymond


### Step 4. Improve 'time' column

In [385]:
# see the phrases presented: replace all numbers with blanks
time_column = audible.time.str.replace(r'[0-9]', '', regex = True)
#keep only unique patterns
time_column.unique()

array([' hrs and  mins', ' hrs', ' hrs and  min', ' hr and  mins',
       ' mins', ' hr', ' hr and  min', ' min', 'Less than  minute'],
      dtype=object)

In [386]:
# Define the conversion function:
def convert_to_minutes(time_duration):
    parts = time_duration.split()
    duration_minutes = 0
    for i in range(0, len(parts), 4):
        if i + 1 < len(parts) and 'hr' in parts[i + 1]:
            duration_minutes += int(parts[i]) * 60
        if i + 3 < len(parts) and 'min' in parts[i + 3]:
            duration_minutes += int(parts[i + 2])
    return duration_minutes

# Apply the function to the dataframe: 
audible['duration_in_minutes'] = audible['time'].apply(convert_to_minutes)   

In [422]:
# check the data type
audible['duration_in_minutes'].dtype

dtype('int64')

In [423]:
# review the data
audible['duration_in_minutes'].describe()

count    87489.000000
mean       389.374904
std        363.609916
min          0.000000
25%        120.000000
50%        360.000000
75%        540.000000
max       8580.000000
Name: duration_in_minutes, dtype: float64

In [389]:
# Once verified that the changes are correct, drop the 'time' column
# Since the values were replaced with the 'duration_in_minutes' column
audible.drop('time', axis=1, inplace=True)

### Step 5. Refactor 'releasedate' column

In [390]:
# Convert the 'date' column to datetime
audible['release_date'] = pd.to_datetime(audible['releasedate'], format = '%d-%m-%y',errors='coerce')

In [391]:
# review the data
audible.head()

,name,author,narrator,releasedate,language,stars,price,duration_in_minutes,release_date
0,Geronimo Stilton #11 & #12,Geronimo Stilton,Bill Lobely,04-08-08,English,5 out of 5 stars34 ratings,468.00,120,2008-08-04
1,The Burning Maze,Rick Riordan,Robbie Daymond,01-05-18,English,4.5 out of 5 stars41 ratings,820.00,780,2018-05-01
2,The Deep End,Jeff Kinney,Dan Russell,06-11-20,English,4.5 out of 5 stars38 ratings,410.00,120,2020-11-06
3,Daughter of the Deep,Rick Riordan,Soneela Nankani,05-10-21,English,4.5 out of 5 stars12 ratings,615.00,660,2021-10-05
4,"The Lightning Thief: Percy Jackson, Book 1",Rick Riordan,Jesse Bernstein,13-01-10,English,4.5 out of 5 stars181 ratings,820.00,600,2010-01-13


In [421]:
# Check the data type
audible['release_date'].info()

<class 'pandas.core.series.Series'>
RangeIndex: 87489 entries, 0 to 87488
Series name: release_date
Non-Null Count  Dtype         
--------------  -----         
87489 non-null  datetime64[ns]
dtypes: datetime64[ns](1)
memory usage: 683.6 KB


In [393]:
# Now that we are confident the data in the 'releasedate' column is stored correctly with the right data type in a new column,
# we can drop the previous one
audible.drop('releasedate', axis=1, inplace=True)

### Step 6. Ensure consistency in 'language' column

In [394]:
# check for unique values
audible.language.unique()

array(['English', 'Hindi', 'spanish', 'german', 'french', 'catalan',
       'swedish', 'italian', 'danish', 'finnish', 'dutch', 'hebrew',
       'russian', 'polish', 'galician', 'afrikaans', 'icelandic',
       'romanian', 'japanese', 'tamil', 'portuguese', 'urdu', 'hungarian',
       'czech', 'bulgarian', 'mandarin_chinese', 'basque', 'korean',
       'arabic', 'greek', 'turkish', 'ukrainian', 'slovene', 'norwegian',
       'telugu', 'lithuanian'], dtype=object)

In [395]:
# Capitalize the first letter in the 'language' column
audible['language'] = audible['language'].str.title()
# Convert the 'language' column to string format
audible['language'] = audible['language'].astype('string')

In [396]:
# check for made changes
audible.language.value_counts()

language
English             61884
German               8295
Spanish              3496
Japanese             3167
Italian              2694
French               2386
Russian              1804
Danish                935
Portuguese            526
Swedish               515
Hindi                 436
Polish                224
Finnish               197
Dutch                 190
Tamil                 161
Catalan               153
Mandarin_Chinese       97
Icelandic              52
Romanian               50
Hungarian              36
Urdu                   34
Afrikaans              28
Czech                  23
Turkish                20
Greek                  18
Arabic                 16
Norwegian              16
Galician               10
Bulgarian               9
Korean                  4
Slovene                 4
Hebrew                  2
Basque                  2
Telugu                  2
Lithuanian              2
Ukrainian               1
Name: count, dtype: Int64

In [426]:
#Check the datatype of the 'language' column
audible['language'].dtype

string[python]

### Step 7. Split 'stars' column into 'stars' and 'rating'

In [398]:
# see the phrases presented
#replace all numbers with blanks
stars_column = audible['stars'].str.replace(r'[0-9]', '', regex = True)
#keep only unique patterns
stars_column.unique()

array([' out of  stars ratings', '. out of  stars ratings',
       'Not rated yet', ' out of  stars rating',
       ' out of  stars, ratings', '. out of  stars, ratings'],
      dtype=object)

In [399]:
audible['stars'].value_counts().get('Not rated yet', 0)

72417

In [400]:
# Convert the 'stars' column to string
audible['stars'] = audible['stars'].astype(str)

# Extract stars and ratings using regular expressions
pattern = r'([\d.]+) out of [\d.]+ stars.*?([\d,]+) ratings'
extracted_data = audible['stars'].str.extract(pattern, expand=True)

# Fill NaN values in the 'ratings' column with 0
extracted_data[1] = extracted_data[1].fillna('0')

# Remove commas and convert to integer
audible['ratings'] = extracted_data[1].str.replace(',', '').astype(float)

# Fill NaN values in the 'stars' column with 0
extracted_data[0] = extracted_data[0].fillna('0')

# Convert to float
audible['stars'] = extracted_data[0].astype(float)


In [401]:
# Check the summary statistics for the 'stars' and 'ratings' columns to ensure the data is appropriate
audible[['stars', 'ratings']].describe()

,stars,ratings
count,87489.000000,87489.000000
mean,0.473888,3.656825
std,1.388756,86.502055
min,0.000000,0.000000
25%,0.000000,0.000000
50%,0.000000,0.000000
75%,0.000000,0.000000
max,5.000000,12573.000000


In [415]:
# review the data type
audible[['stars', 'ratings']].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 87489 entries, 0 to 87488
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   stars    87489 non-null  float64
 1   ratings  87489 non-null  float64
dtypes: float64(2)
memory usage: 1.3 MB


In [403]:
# review the data
audible[['stars', 'ratings']].head(10)

,stars,ratings
0,5.0,34.0
1,4.5,41.0
2,4.5,38.0
3,4.5,12.0
4,4.5,181.0
5,5.0,72.0
6,5.0,11.0
7,5.0,50.0
8,5.0,5.0
9,5.0,58.0


### Step 8. Investigate 'price' column format

In [404]:
# Convert the 'price' column to string
audible['price'] = audible['price'].astype(str)

# Extract numeric values with or without comma separators using regular expression
audible['extracted_price'] = audible['price'].str.extract(r'([\d,]+\.\d+|\d+)')

# Remove commas from the extracted values
audible['extracted_price'] = audible['extracted_price'].str.replace(',', '')

# Replace all NaN values in the 'extracted_price' column with 0
audible['extracted_price'].fillna(0, inplace=True)

# Convert the 'extracted_price' column to float
audible['extracted_price'] = audible['extracted_price'].astype(float)


In [425]:
# review the data
audible.price.describe()

count    87489.000000
mean       559.009246
std        336.096642
min          0.000000
25%        268.000000
50%        585.000000
75%        755.000000
max       7198.000000
Name: price, dtype: float64

In [406]:
# Drop the 'price' column
audible.drop('price', axis=1, inplace=True)

# Rename 'extracted_price' to 'priudibce'
audible.rename(columns={'extracted_price': 'price'}, inplace=True)


### Step 9. Review the data

In [407]:
audible.head(10)

,name,author,narrator,language,stars,duration_in_minutes,release_date,ratings,price
0,Geronimo Stilton #11 & #12,Geronimo Stilton,Bill Lobely,English,5.0,120,2008-08-04,34.0,468.0
1,The Burning Maze,Rick Riordan,Robbie Daymond,English,4.5,780,2018-05-01,41.0,820.0
2,The Deep End,Jeff Kinney,Dan Russell,English,4.5,120,2020-11-06,38.0,410.0
3,Daughter of the Deep,Rick Riordan,Soneela Nankani,English,4.5,660,2021-10-05,12.0,615.0
4,"The Lightning Thief: Percy Jackson, Book 1",Rick Riordan,Jesse Bernstein,English,4.5,600,2010-01-13,181.0,820.0
5,The Hunger Games: Special Edition,Suzanne Collins,Tatiana Maslany,English,5.0,600,2018-10-30,72.0,656.0
6,Quest for the Diamond Sword,Winter Morgan,Luke Daniels,English,5.0,120,2014-11-25,11.0,233.0
7,The Dark Prophecy,Rick Riordan,Robbie Daymond,English,5.0,720,2017-05-02,50.0,820.0
8,Merlin Mission Collection,Mary Pope Osborne,Mary Pope Osborne,English,5.0,600,2017-05-02,5.0,1256.0
9,The Tyrant’s Tomb,Rick Riordan,Robbie Daymond,English,5.0,780,2019-09-24,58.0,820.0


In [424]:
audible.describe(include='all')

,name,author,narrator,language,stars,duration_in_minutes,release_date,ratings,price
count,87489,87489,87489,87489,87489.000000,87489.000000,87489,87489.000000,87489.000000
unique,82767,48374,29717,36,NaN,NaN,NaN,NaN,NaN
top,The Art of War,"矢島雅弘,石橋遊",anonymous,English,NaN,NaN,NaN,NaN,NaN
freq,20,874,1034,61884,NaN,NaN,NaN,NaN,NaN
mean,NaN,NaN,NaN,NaN,0.473888,389.374904,2018-06-23 22:53:20.411480064,3.656825,559.009246
min,NaN,NaN,NaN,NaN,0.000000,0.000000,1998-12-27 00:00:00,0.000000,0.000000
25%,NaN,NaN,NaN,NaN,0.000000,120.000000,2016-09-06 00:00:00,0.000000,268.000000
50%,NaN,NaN,NaN,NaN,0.000000,360.000000,2020-02-06 00:00:00,0.000000,585.000000
75%,NaN,NaN,NaN,NaN,0.000000,540.000000,2021-08-17 00:00:00,0.000000,755.000000
max,NaN,NaN,NaN,NaN,5.000000,8580.000000,2025-11-14 00:00:00,12573.000000,7198.000000


In [427]:
audible.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 87489 entries, 0 to 87488
Data columns (total 9 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   name                 87489 non-null  object        
 1   author               87489 non-null  object        
 2   narrator             87489 non-null  object        
 3   language             87489 non-null  string        
 4   stars                87489 non-null  float64       
 5   duration_in_minutes  87489 non-null  int64         
 6   release_date         87489 non-null  datetime64[ns]
 7   ratings              87489 non-null  float64       
 8   price                87489 non-null  float64       
dtypes: datetime64[ns](1), float64(3), int64(1), object(3), string(1)
memory usage: 6.0+ MB


## Conclusion

After a comprehensive data cleaning process, the dataset has been transformed into a tidy and well-structured format. 
- Prefixes in the 'author' and 'narrator' columns have been addressed, and names are now formatted consistently. 
- The 'time' and 'releasedate' columns have been processed to facilitate further analysis.
- Inconsistent language capitalization has been corrected.
- Additionally, the 'stars' column has been split into 'stars' and 'ratings' for better numerical representation.
- The 'price' column has been converted to a numeric format.

The dataset is now ready for in-depth analysis and insights.